# Automate Grading AMATH 301
The purpose of this notebook is to combine grades from Matlab Grader and Gradescope. It will then add the scores to the canvas gradebook CSV to import. Certain lines will need to be edited each time it is used. The "reference" CSV is needed to link students by email and ID number. Before each code block that needs edited is a cell that outlines which lines need to be changed under "TO DO". 

### Debugging Notes: 
1. Make sure in references that there are no spaces in front of the ID numbers.
3. I also ran into some data type issues, but changing everything to float seemed to fix it.

TO DO: 
1. Change the path to the location where the CSV files are located. 

In [1]:
import pandas as pd;
mat_grader = pd.read_csv('/Users/shannondow/Desktop/Automate_Grading/MatlabGrader_hw1.csv')
gradescope = pd.read_csv('/Users/shannondow/Desktop/Automate_Grading/Gradescope_hw1.csv')
canvas = pd.read_csv('/Users/shannondow/Desktop/Automate_Grading/Canvas_hw1.csv')
reference = pd.read_csv('/Users/shannondow/Desktop/Automate_Grading/References.csv')

In [2]:
mat_grader = mat_grader.sort_values(by = ['Student Email'])
mat_grader = mat_grader[[ 'Student Email', 'Tests Passed','Total Tests','Problem Title']]
gradescope = gradescope[['SID','Total Score']]
gradescope = gradescope.rename(columns = {'Total Score': 'GS Tot'})
reference = reference.rename(columns = {'StudentNo': 'SID'})
reference = reference.rename(columns = {'Email':'Student Email'})

TO DO: 
1. Edit the homework number
2. Edit mgw (Matlab Grader Weight) This is the weight of each question.
3. Edit them number of matlab grader questions 
4. Create an entry in the dictionary (mgs) for each matlab grader question

In [3]:
hw_number = 1;
mgw = 5
num_mg_questions = 2

mg1 = mat_grader[mat_grader['Problem Title'] == 'HW1: Problem 1']
mg2 = mat_grader[mat_grader['Problem Title'] == 'HW1: Problem 2']  

mgs = {1:mg1,2:mg2}

In [4]:
for key in mgs:
    prob_name = 'mg_q'+ str(key)
    mgs[key] = mgs[key].rename(columns = {'Tests Passed':prob_name})
    mgs[key] = mgs[key][[ 'Student Email', prob_name]]
    reference = reference.merge(mgs[key], on = 'Student Email',how = 'left')
    reference[prob_name] = reference[prob_name].fillna(0)
tot_sum = 0;
for key in mgs:
    prob_name = 'mg_q'+ str(key)
    tot_sum += reference[prob_name]
    
reference['mg_tot'] = tot_sum*mgw

In [5]:
gradescope['SID'] = gradescope['SID'].astype(float)
reference['SID'] = reference['SID'].astype(float)
reference = reference.merge(gradescope, on = 'SID',how = 'left')
reference["GS Tot"] = reference["GS Tot"].fillna(0)
reference['TOTAL'] = (reference['mg_tot']+ reference['GS Tot'])

TO DO: 
1. change col_edit. This is the column of the canvas csv file that will be edited. You will need to look at the CSV and recall that python begins indexing at 0

In [6]:
col_edit = 6;
i = 2;
while i < canvas.shape[0]:
    canvas.iloc[i,col_edit] = 0
    i = i+1
i = 2
while i < canvas.shape[0]:
    for j in range(0,reference.shape[0]):
        if canvas.iloc[i,3] == reference.iloc[j,1]:
            canvas.iloc[i,col_edit] = reference.iloc[j,(7+num_mg_questions)]
    i = i+1

Verify:

In [7]:
pd.set_option('display.max_columns', canvas.shape[1]+1)
canvas

,Student,ID,SIS User ID,SIS Login ID,Section,Homework 0 (5327434),Homework 1 (5327437),Homework 2 (5431255),Activity 1 (5327424),Beginning of Quarter Survey (5327426),Activity 2 (5327420),Activity 3 (5327416),Homework Current Score,Homework Unposted Current Score,Homework Final Score,Homework Unposted Final Score,Activities/Quizzes Current Score,Activities/Quizzes Unposted Current Score,Activities/Quizzes Final Score,Activities/Quizzes Unposted Final Score,Midterm Current Score,Midterm Unposted Current Score,Midterm Final Score,Midterm Unposted Final Score,Final Exam Current Score,Final Exam Unposted Current Score,Final Exam Final Score,Final Exam Unposted Final Score,Imported Assignments Current Score,Imported Assignments Unposted Current Score,Imported Assignments Final Score,Imported Assignments Unposted Final Score,Current Score,Unposted Current Score,Final Score,Unposted Final Score
0,NaN,NaN,NaN,NaN,NaN,Manual Posting,Manual Posting,Manual Posting,Manual Posting,Manual Posting,Manual Posting,Manual Posting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Points Possible,NaN,NaN,NaN,NaN,0,100,100,10,3,10,10,(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only)
2,"Felten, Anna",1234563.0,796C2D5CA1DB557115E7629EBD314831,feltan,AMATH 301 B,NaN,93,NaN,10,3,10,10,NaN,NaN,0,0,100,100,69.7,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20
3,"Ellis, Andrew",9922782.0,796C2D5CA1DB557115E7629EBD314831,drewb,AMATH 301 B,NaN,76,NaN,10,3,10,10,NaN,NaN,0,0,100,100,69.7,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20
4,"Tubert, Nik",1836593.0,796C2D5CA1DB557115E7629EBD314831,skimnik,AMATH 301 B,NaN,95,NaN,10,3,10,10,NaN,NaN,0,0,100,100,69.7,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20
5,"Serrano, Caroline",1695437.0,796C2D5CA1DB557115E7629EBD314831,caroser,AMATH 301 B,NaN,69,NaN,10,3,10,10,NaN,NaN,0,0,100,100,69.7,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20
6,"Hamada, Evan",9087642.0,796C2D5CA1DB557115E7629EBD314831,mint,AMATH 301 B,NaN,100,NaN,10,3,10,10,NaN,NaN,0,0,100,100,69.7,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20
7,"Calhoun, Megan",2737462.0,796C2D5CA1DB557115E7629EBD314831,meg23,AMATH 301 B,NaN,95,NaN,10,3,10,10,NaN,NaN,0,0,100,100,69.7,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20
8,"Brown, Ashlyn",2839403.0,796C2D5CA1DB557115E7629EBD314831,hash56,AMATH 301 B,NaN,75,NaN,10,3,10,10,NaN,NaN,0,0,100,100,69.7,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20
9,"Chaffee, Sami",8392817.0,796C2D5CA1DB557115E7629EBD314831,samsam9,AMATH 301 B,NaN,90,NaN,10,3,10,10,NaN,NaN,0,0,100,100,69.7,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20


TO DO: 
1. Save the updated CSV to a specified location with a desired name: 

In [8]:
canvas.to_csv('/Users/shannondow/Desktop/Automate_Grading/HW_update.csv')

NOTE: 
1. In my experience, canvas gradebook does not accept the edited CSV as an import. Instead, I copy the only edited column into the exported gradebook for import.